<a href="https://colab.research.google.com/github/Prabh-84/Vision-transformer-for-pest-Detection/blob/main/pest_vision_trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install timm


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully 

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/pest_dataset.zip"  # 👈 replace with actual path
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted!")


✅ Dataset extracted!


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dir = '/content/dataset/train'
test_dir = '/content/dataset/test'

train_data = ImageFolder(train_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

num_classes = len(train_data.classes)


In [ ]:
print(f"📁 Total training images: {len(train_data)}")
print(f"🧪 Total test images: {len(test_data)}")


📁 Total training images: 4395
🧪 Total test images: 1099


In [ ]:
import timm
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ViT base model and modify for classification
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, num_classes)
model = model.to(device)

print("✅ Vision Transformer model is ready!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

✅ Vision Transformer model is ready!


In [ ]:
import torch.optim as optim

# Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm

train_accuracies = []
test_accuracies = []

epochs = 15

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        if outputs.dim() > 2:
            outputs = outputs.view(outputs.size(0), -1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

    train_acc = 100 * correct / total
    train_accuracies.append(train_acc)
    print(f"✅ Epoch {epoch+1} | Train Acc: {train_acc:.2f}% | Loss: {running_loss:.4f}")

    # Evaluation
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.dim() > 2:
                outputs = outputs.view(outputs.size(0), -1)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    test_acc = 100 * test_correct / test_total
    test_accuracies.append(test_acc)
    print(f"🧪 Test Accuracy after Epoch {epoch+1}: {test_acc:.2f}%\n")


✅ Epoch 1 | Train Acc: 85.60% | Loss: 65.7793
🧪 Test Accuracy after Epoch 1: 89.90%



✅ Epoch 2 | Train Acc: 94.70% | Loss: 23.4262
🧪 Test Accuracy after Epoch 2: 90.72%



✅ Epoch 3 | Train Acc: 96.59% | Loss: 14.2630
🧪 Test Accuracy after Epoch 3: 87.08%



✅ Epoch 4 | Train Acc: 96.61% | Loss: 14.2613
🧪 Test Accuracy after Epoch 4: 88.17%



✅ Epoch 5 | Train Acc: 97.47% | Loss: 10.8806
🧪 Test Accuracy after Epoch 5: 88.08%



...

✅ Epoch 11 | Train Acc: 98.59% | Loss: 6.3687
🧪 Test Accuracy after Epoch 11: 86.17%



✅ Epoch 12 | Train Acc: 99.32% | Loss: 3.8113
🧪 Test Accuracy after Epoch 12: 86.44%



✅ Epoch 13 | Train Acc: 99.02% | Loss: 4.0323
🧪 Test Accuracy after Epoch 13: 88.35%



✅ Epoch 14 | Train Acc: 97.84% | Loss: 9.9269
🧪 Test Accuracy after Epoch 14: 82.53%



✅ Epoch 15 | Train Acc: 98.38% | Loss: 7.8061
🧪 Test Accuracy after Epoch 15: 85.26%



In [ ]:
test_accuracies = [89.90,90.72,87.08,88.17,88.08,86.08,87.08,83.99,85.26,86.08,86.17,86.44,88.35,82.53,85.26]
average_test_accuracy = sum(test_accuracies) / len(test_accuracies)
print(f"📊 Average Test Accuracy: {average_test_accuracy:.2f}%")


📊 Average Test Accuracy: 86.75%


In [ ]:
save_path = "/content/drive/MyDrive/vit_leaf_model.pth"
torch.save(model.state_dict(), save_path)
print("💾 Model saved to:", save_path)


💾 Model saved to: /content/drive/MyDrive/vit_leaf_model.pth


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving test_pest.jpg to test_pest.jpg


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Get the uploaded image name
image_path = list(uploaded.keys())[0]

# Load and preprocess image
image = Image.open(image_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

img_tensor = transform(image).unsqueeze(0).to(device)

# Predict
model.eval()
with torch.no_grad():
    output = model(img_tensor)
    if output.dim() > 2:
        output = output.view(output.size(0), -1)
    _, pred = output.max(1)

predicted_class = train_data.classes[pred.item()]
print(f"🔍 Predicted Class: {predicted_class}")


🔍 Predicted Class: weevil
